In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df =  pd.read_csv('/content/train_processed (2).csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/train_processed (2).csv'

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

class PreProcessingWithFE:
    def __init__(self):
        self.scaler = None
        self.features = ['price_change_pct', 'vol_change_pct', 'price_range', 'volatility', 'RSI']

    def preprocess(self, df):

        # Drop missing values
        df = df.dropna()

        # Convert to datetime format
        df['Date'] = pd.to_datetime(df['Date'])



        # Sort by date
        df = df.sort_values('Date')

        # Additional features
        df['price_range'] = df['High'] - df['Low']
        df['price_change'] = df['Close'] - df['Open']
        df['price_change_pct'] = np.where(df['Open'] != 0,
                                          (df['Close'] - df['Open']) / df['Open'] * 100,
                                          0)
        df['vol_change_pct'] = df['Volume'].pct_change() * 100

        # Moving averages
        df['MA5'] = df['Close'].rolling(window=5, min_periods=1).mean()
        df['MA20'] = df['Close'].rolling(window=20, min_periods=1).mean()

        # Volatility (20-day rolling standard deviation)
        df['volatility'] = df['Close'].rolling(window=20).std()

        # RSI (Relative Strength Index)
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        # Clip extreme values for selected features
        for feature in self.features:
            df[feature] = df[feature].clip(-1e10, 1e10)

        # Standardize selected features
        self.scaler = StandardScaler()
        df_scaled = pd.DataFrame(
            self.scaler.fit_transform(df[self.features]),
            columns=self.features,
            index=df.index
        )

        return df, df_scaled


In [ ]:
preprocessor = PreProcessingWithFE()

data, train_data = preprocessor.preprocess(df)

In [ ]:
data.info()

In [ ]:
data.head()